In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm.auto import tqdm

import pandas as pd

import numpy as np

from torch.utils.data import Dataset, DataLoader

In [4]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
try:
    df = pd.read_csv("ctx_quest.csv")
except:
    df = pd.read_csv("/Users/lilyakhoang/input/question_generation/ctx_quest.csv")

df.head()

,Unnamed: 0,context,question
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
2,2,"Managed by her father, Mathew Knowles, the gro...",When did Beyonce leave Destiny's Child and bec...
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?


In [10]:
# потом можете добавить свою предобработку

def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [8]:
all_text_data = []
all_text_data.extend(list(df['context']))
all_text_data.extend(list(df['question']))
all_text_data = list(set(all_text_data))
cleaned_text_data = [x for x in all_text_data if 'float' not in str(type(x))]
cleaned_text_data[:2]

['When did the U.N. vote to adopt the Sustainable Development Goals?',
 "In what case can a referee's decision be overturned?"]

In [11]:
word2freq = {}
lengths = []

for text in tqdm(cleaned_text_data):
  
    words = process_text(text)
    
    lengths.append(len(words))
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

In [12]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
local_path = "/Users/lilyakhoang/input/glove.6B/glove.6B.50d.txt"
this_folder_path = "glove.6B.50d.txt"

word2index = {'PAD': 0}
vectors = []
   
try:
    lines = open(local_path)
except:
    lines = open(this_folder_path)
    
embedding_dim = 50
# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))
progress_bar = tqdm(desc='Read word2vec', total=400000)

for line in lines:
    current_word = line.split()[0]
    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        # current_vectors = current_parts[-embedding_dim:]
        current_vectors = line.split()[1:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

vectors = np.concatenate(vectors)

lines.close()

In [51]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, vectors=vectors):#input_size
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [15]:
class DecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, vectors):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        # self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, debug = False):
        if debug == True:
          print("===FORWARD_DECODER===")
          print("input.shape {}, hidden.shape {}".format(input.shape,hidden.shape ))
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [37]:
class WordData(Dataset):
    def __init__(self, context , word2index, context_sequence_length=80, 
                 pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        
        self.word2index = word2index
        self.context_sequence_length = context_sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(context, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
    
        words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for ctx_sent in data_iterator:
            ctx = self.process_text(ctx_sent)
            indexed_ctx = self.indexing(ctx)
            self.x_data.append(indexed_ctx)
    
    def indexing(self, tokenized_text):
        indexes = []
        for word in tokenized_text:
          if word in self.word2index:
            indexes.append(self.word2index[word])
        return indexes
    
    def padding(self, sequence):
        count = 0 
        paded_seq = []
        for seq_el in sequence:
              paded_seq.append(seq_el)
              count += 1
              if count >= self.context_sequence_length: break
        if count < self.context_sequence_length:
          for ind in range(count, self.context_sequence_length):
            paded_seq.append(self.pad_index)
        return paded_seq
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()

        return x

In [38]:
dataset = WordData(context, word2index)

In [39]:
dataset.__getitem__(0)

tensor([33972, 33404, 16932,    11,  3127,   270,  8242,    11, 36992,    11,
          200,    24,    23,   375,   437,   405,     2,  2852,    24,    14,
           28,   137,  2195,     2,  9315,     2,   380,  1913,     6,  2844,
            3,   375,     6,  1059,     7,  1971,     2,   736,     2,    66,
         1767,     7,   904,  4027,     6,  5264,  5557,    19,     8,   951,
            2,     6,   480,     5,  3101,     7,     1,   284,  2222,    19,
          406,  2195,     4,  1886,  1146,  1535,  1728,    11,   126,  9697,
           56,  1513,   951,     3,  1743,    21,    69,   623,     2, 23818])

In [41]:
data_loader = DataLoader(dataset, drop_last = True)
for x,in data_loader:
    print(x.shape)
    break

torch.Size([80])


In [52]:
encoder_no_att = torch.load("/Users/lilyakhoang/Desktop/dl_project_out_of_git/Encoder_no_attention_5_epochs_2.7", map_location='cpu')

In [53]:
decoder_no_att = torch.load("/Users/lilyakhoang/Desktop/dl_project_out_of_git/Decoder_no_attention_5_epochs_2.7", map_location='cpu')

In [58]:
SOS_token = 0


In [70]:
index2word = {word2index[k]:k for k in word2index}

In [73]:
index2word[2423]

'tourism'

In [98]:
context_trial = ["""Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, 
Texas, she performed in various singing and dancing competitions as a child,
and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's
Child. Managed by her father, Mathew Knowles, the group became one of the world's 
best-selling girl groups of all time."""]
           
           

In [78]:
context_trial = ["""Texas, she performed in various singing and dancing competitions as a child,
and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child."""]
           
           

In [100]:
def predict(context_list, encoder, decoder):
    dataset = WordData(context_list, word2index)
    data_loader = DataLoader(dataset, drop_last = True)
    for context_tensor in data_loader:
        context_tensor_words = [index2word[int(index)] for index in context_tensor[0] if index != 0]
        context_tensor_words = ' '.join(context_tensor_words)
        print("CONTEXT ==> ",context_tensor_words)
        output_question = ''
        with torch.no_grad():
            encoder_hidden = encoder.initHidden()
            encoder_output, encoder_hidden = encoder(context_tensor, encoder_hidden)
#             print(encoder_output.shape,encoder_hidden.shape )
            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden
            for di in range(20):
#                 print("iterate_over_target {}th time, decoder_input.shape {}".format(di, decoder_input.shape))
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#                 print("decoder_output.shape {}, decoder_hidden.shape{}".format(decoder_output.shape, decoder_hidden.shape))
                word_index = int(torch.argmax(decoder_output))
#                 print(word_index)
                output_question += ' ' + index2word[word_index]
                if word_index == SOS_token:
                    print("\nQUESTION ==> ", output_question)
                    break
                    
predict(context_trial, encoder_no_att, decoder_no_att)        


CONTEXT ==>  beyoncé giselle knowles - carter / bee - yon - say ) ( born september 4 , 1981 ) is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r & b girl - group destiny ' s child . managed by her father , mathew

QUESTION ==>   what PAD
